## Questão 01

In [96]:
import numpy as np
import random
from sklearn.model_selection import train_test_split

In [97]:
response_vectors = np.array([
    [1, -1, -1, -1, -1, -1, -1, -1],
    [-1, 1, -1, -1, -1, -1, -1, -1],
    [-1, -1, 1, -1, -1, -1, -1, -1],
    [-1, -1, -1, 1, -1, -1, -1, -1],
    [-1, -1, -1, -1, 1, -1, -1, -1],
    [-1, -1, -1, -1, -1, 1, -1, -1],
    [-1, -1, -1, -1, -1, -1, 1, -1],
    [-1, -1, -1, -1, -1, -1, -1, 1]
], dtype=np.float32)

data_X = []
data_y = []

for i in range(10000):
    instance = np.array([random.randint(0,1), random.randint(0,1), random.randint(0,1)])
    label = instance[0]*4 + instance[1]*2 + instance[2]
    noise = np.array([random.uniform(-0.1, 0.1), random.uniform(-0.1, 0.1), random.uniform(-0.1, 0.1)])

    data_X.append(instance+noise)
    data_y.append(label)

data_X = np.array(data_X, dtype=np.float32)
data_y = np.array(data_y, dtype=np.float32)

X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.2, random_state=10)

In [98]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score

clf = Perceptron(tol=1e-3, random_state=10, validation_fraction=0.2, early_stopping=True, verbose=True)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

-- Epoch 1
Norm: 3.23, NNZs: 3, Bias: 1.000000, T: 6400, Avg. loss: 0.000507
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 3.23, NNZs: 3, Bias: 1.000000, T: 12800, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 3.23, NNZs: 3, Bias: 1.000000, T: 19200, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 4
Norm: 3.23, NNZs: 3, Bias: 1.000000, T: 25600, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 5
Norm: 3.23, NNZs: 3, Bias: 1.000000, T: 32000, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 6
Norm: 3.23, NNZs: 3, Bias: 1.000000, T: 38400, Avg. loss: 0.000000
Total training time: 0.01 seconds.
Convergence after 6 epochs took 0.01 seconds
-- Epoch 1
Norm: 3.12, NNZs: 3, Bias: -1.000000, T: 6400, Avg. loss: 0.000777
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 3.12, NNZs: 3, Bias: -1.000000, T: 12800, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 3.12, NNZs: 3, Bias: -1.000000, T:

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished


In [99]:
## plotar gráfico tridimensional com os pontos e suas respectivas classes usando o plotly
import plotly.graph_objects as go
fig = go.Figure(data=[go.Scatter3d(x=X_test[:,0], y=X_test[:,1], z=X_test[:,2], mode='markers', marker=dict(size=3, color=y_pred))])
fig.show()

## Questão 02

In [100]:
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import random
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [101]:
# Função de treinamento
def train(model, criterion, optimizer, train_loader, val_loader, epochs=100):
    train_losses = []
    val_losses = []
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1))
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        train_loss = running_loss / len(train_loader)
        train_losses.append(train_loss)

        val_loss = evaluate(model, criterion, val_loader)
        val_losses.append(val_loss)
        
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

    return train_losses, val_losses


# Função de avaliação
def evaluate(model, criterion, val_loader):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1))
            running_loss += loss.item()
    return running_loss / len(val_loader)


### a)

In [102]:
n = 10000
data_X = []
data_y = []

for i in range(n):
    instance = np.array([random.uniform(-5,5), random.uniform(-5,5)])

    if instance[0]*instance[1] > 0:
        label = 1
    elif instance[0]*instance[1] < 0:
        label = 0
    else:
        continue

    data_X.append(instance)
    data_y.append(label)


data_X = np.array(data_X)
data_y = torch.tensor(data_y, dtype=torch.float32)

In [103]:
# Dividir os dados em treino, validação e teste usando train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.2, random_state=10)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=10)

# Converter para tensores
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_val_tensor = torch.tensor(x_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Definir o modelo da rede neural
class PerceptronXOR(nn.Module):
    def __init__(self):
        super(PerceptronXOR, self).__init__()
        self.fc1 = nn.Linear(2, 4)  # Camada de entrada
        self.fc2 = nn.Linear(4, 1)  # Camada de saída

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

# Instanciar o modelo, função de perda e otimizador
model = PerceptronXOR()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Definir o tamanho do batch e criar os DataLoader
batch_size = 32
train_loader = DataLoader(TensorDataset(x_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(x_val_tensor, y_val_tensor), batch_size=batch_size)

# Treinar o modelo
train_losses, val_losses = train(model, criterion, optimizer, train_loader, val_loader, epochs=100)

# Avaliar o modelo nos dados de teste
test_loader = DataLoader(TensorDataset(x_test_tensor, y_test_tensor), batch_size=batch_size)
test_loss = evaluate(model, criterion, test_loader)
print(f"\nTest Loss: {test_loss:.4f}")

# Cálculo da acurácia
model.eval()
with torch.no_grad():
    outputs = model(x_test_tensor)
    predicted = torch.round(outputs)
    accuracy = (predicted == y_test_tensor.unsqueeze(1)).sum().item() / predicted.size(0)
    print(f"Acurácia: {accuracy:.2f}")


C:\Users\jahon\AppData\Local\Temp\ipykernel_5980\2157785410.py:7: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

C:\Users\jahon\AppData\Local\Temp\ipykernel_5980\2157785410.py:9: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

C:\Users\jahon\AppData\Local\Temp\ipykernel_5980\2157785410.py:11: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 1/100, Train Loss: 0.6999, Val Loss: 0.6616
Epoch 2/100, Train Loss: 0.6223, Val Loss: 0.5738
Epoch 3/100, Train Loss: 0.5201, Val Loss: 0.4780
Epoch 4/100, Train Loss: 0.4374, Val Loss: 0.4126
Epoch 5/100, Train Loss: 0.3803, Val Loss: 0.3647
Epoch 6/100, Train Loss: 0.3385, Val Loss: 0.3282
Epoch 7/100, Train Loss: 0.3059, Val Loss: 0.2988
Epoch 8/100, Train Loss: 0.2799, Val Loss: 0.2754
Epoch 9/100, Train Loss: 0.2586, Val Loss: 0.2554
Epoch 10/100, Train Loss: 0.2405, Val Loss: 0.2384
Epoch 11/100, Train Loss: 0.2247, Val Loss: 0.2234
Epoch 12/100, Train Loss: 0.2112, Val Loss: 0.2109
Epoch 13/100, Train Loss: 0.1992, Val Loss: 0.1992
Epoch 14/100, Train Loss: 0.1887, Val Loss: 0.1888
Epoch 15/100, Train Loss: 0.1791, Val Loss: 0.1795
Epoch 16/100, Train Loss: 0.1706, Val Loss: 0.1710
Epoch 17/100, Train Loss: 0.1626, Val Loss: 0.1635
Epoch 18/100, Train Loss: 0.1558, Val Loss: 0.1564
Epoch 19/100, Train Loss: 0.1495, Val Loss: 0.1508
Epoch 20/100, Train Loss: 0.1436, Val Lo

In [104]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(train_losses)), y=train_losses, mode='lines', name='Train Loss'))
fig.add_trace(go.Scatter(x=np.arange(len(val_losses)), y=val_losses, mode='lines', name='Val Loss'))
fig.update_layout(height=600, title='Train and Val Loss', xaxis_title='Epoch', yaxis_title='Loss')
fig.add_trace(go.Scatter(x=[len(train_losses)], y=[test_loss], mode='markers', marker=dict(color='orange', size=10), name='Test Loss'))
fig.show()

### b)

In [105]:
n=10000

data_X = []
data_y = []

for i in range(n):
    instance = np.array([random.uniform(1,10), random.uniform(0,1)])
    if np.log10(instance[0]) > instance[1]:
        label = 0
    else:
        label = 1

    data_X.append(instance)
    data_y.append(label)

x = np.array(data_X)
y = torch.tensor(data_y, dtype=torch.float32)

In [106]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=10)

x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_val_tensor = torch.tensor(x_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Definir o modelo da rede neural
class PerceptronLog(nn.Module):
    def __init__(self):
        super(PerceptronLog, self).__init__()
        self.fc1 = nn.Linear(2, 4)  # Camada de entrada
        self.fc2 = nn.Linear(4, 1)  # Camada de saída

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

# Instanciar o modelo, função de perda e otimizador
model = PerceptronLog()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Definir o tamanho do batch e criar os DataLoader
batch_size = 32
train_loader = DataLoader(TensorDataset(x_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(x_val_tensor, y_val_tensor), batch_size=batch_size)

# Treinar o modelo
train_losses, val_losses = train(model, criterion, optimizer, train_loader, val_loader, epochs=100)

# Avaliar o modelo nos dados de teste
test_loader = DataLoader(TensorDataset(x_test_tensor, y_test_tensor), batch_size=batch_size)
test_loss = evaluate(model, criterion, test_loader)
print(f"\nTest Loss: {test_loss:.4f}")

model.eval()
with torch.no_grad():
    outputs = model(x_test_tensor)
    predicted = torch.round(outputs)
    accuracy = (predicted == y_test_tensor.unsqueeze(1)).sum().item() / predicted.size(0)
    print(f"Acurácia: {accuracy:.2f}")

C:\Users\jahon\AppData\Local\Temp\ipykernel_5980\2345171390.py:5: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

C:\Users\jahon\AppData\Local\Temp\ipykernel_5980\2345171390.py:7: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

C:\Users\jahon\AppData\Local\Temp\ipykernel_5980\2345171390.py:9: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 1/100, Train Loss: 0.6597, Val Loss: 0.5876
Epoch 2/100, Train Loss: 0.5188, Val Loss: 0.4594
Epoch 3/100, Train Loss: 0.4144, Val Loss: 0.3818
Epoch 4/100, Train Loss: 0.3514, Val Loss: 0.3322
Epoch 5/100, Train Loss: 0.3088, Val Loss: 0.2970
Epoch 6/100, Train Loss: 0.2783, Val Loss: 0.2683
Epoch 7/100, Train Loss: 0.2534, Val Loss: 0.2454
Epoch 8/100, Train Loss: 0.2314, Val Loss: 0.2248
Epoch 9/100, Train Loss: 0.2143, Val Loss: 0.2081
Epoch 10/100, Train Loss: 0.1989, Val Loss: 0.1942
Epoch 11/100, Train Loss: 0.1867, Val Loss: 0.1814
Epoch 12/100, Train Loss: 0.1761, Val Loss: 0.1714
Epoch 13/100, Train Loss: 0.1669, Val Loss: 0.1631
Epoch 14/100, Train Loss: 0.1601, Val Loss: 0.1559
Epoch 15/100, Train Loss: 0.1529, Val Loss: 0.1497
Epoch 16/100, Train Loss: 0.1480, Val Loss: 0.1435
Epoch 17/100, Train Loss: 0.1436, Val Loss: 0.1391
Epoch 18/100, Train Loss: 0.1392, Val Loss: 0.1351
Epoch 19/100, Train Loss: 0.1360, Val Loss: 0.1321
Epoch 20/100, Train Loss: 0.1329, Val Lo

In [107]:
## plot dos pontos de teste e a função log10(x) no intervalo [1,10]

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_test[:,0], y=x_test[:,1], mode='markers', marker=dict(color=y_test, colorscale='Viridis', size=5)))
x = np.linspace(1, 10, 100)
y = np.log10(x)
fig.add_trace(go.Scatter(x=x, y=y, mode='lines', line=dict(color='orange', width=3)))
fig.show()

In [108]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(train_losses)), y=train_losses, mode='lines', name='Train Loss'))
fig.add_trace(go.Scatter(x=np.arange(len(val_losses)), y=val_losses, mode='lines', name='Val Loss'))
fig.update_layout(height=600, title='Train and Val Loss', xaxis_title='Epoch', yaxis_title='Loss')
fig.add_trace(go.Scatter(x=[len(train_losses)], y=[test_loss], mode='markers', marker=dict(color='orange', size=10), name='Test Loss'))
fig.show()

### c)

In [118]:
n=10000

data_X = []
data_y = []

def f(x):
    return 10*x**5 + 5*x**4 + 2*x**3 - 0.5*x**2 + 3*x + 2

for i in range(n):
    instance = np.array([random.uniform(0,5), random.uniform(-5000,5000)])
    if f(instance[0]) > instance[1]:
        label = 0
    else:
        label = 1

    data_X.append(instance)
    data_y.append(label)

x = np.array(data_X)
y = torch.tensor(data_y, dtype=torch.float32)

In [119]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=10)

x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_val_tensor = torch.tensor(x_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Definir o modelo da rede neural
class PerceptronPol(nn.Module):
    def __init__(self):
        super(PerceptronPol, self).__init__()
        self.fc1 = nn.Linear(2, 128)  # Camada de entrada
        self.fc2 = nn.Linear(128, 128)  # Camada oculta
        self.fc3 = nn.Linear(128, 1)  # Camada de saída

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

# Instanciar o modelo, função de perda e otimizador
model = PerceptronPol()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Definir o tamanho do batch e criar os DataLoader
batch_size = 32
train_loader = DataLoader(TensorDataset(x_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(x_val_tensor, y_val_tensor), batch_size=batch_size)

# Treinar o modelo
train_losses, val_losses = train(model, criterion, optimizer, train_loader, val_loader, epochs=100)

# Avaliar o modelo nos dados de teste
test_loader = DataLoader(TensorDataset(x_test_tensor, y_test_tensor), batch_size=batch_size)
test_loss = evaluate(model, criterion, test_loader)
print(f"\nTest Loss: {test_loss:.4f}")

model.eval()
with torch.no_grad():
    outputs = model(x_test_tensor)
    predicted = torch.round(outputs)
    accuracy = (predicted == y_test_tensor.unsqueeze(1)).sum().item() / predicted.size(0)
    print(f"Acurácia: {accuracy:.2f}")

C:\Users\jahon\AppData\Local\Temp\ipykernel_5980\1659026911.py:5: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

C:\Users\jahon\AppData\Local\Temp\ipykernel_5980\1659026911.py:7: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

C:\Users\jahon\AppData\Local\Temp\ipykernel_5980\1659026911.py:9: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Epoch 1/100, Train Loss: 21.9847, Val Loss: 21.4506
Epoch 2/100, Train Loss: 26.5385, Val Loss: 28.2513
Epoch 3/100, Train Loss: 27.9515, Val Loss: 28.2509
Epoch 4/100, Train Loss: 27.9670, Val Loss: 28.2504
Epoch 5/100, Train Loss: 27.8827, Val Loss: 28.2498
Epoch 6/100, Train Loss: 27.9151, Val Loss: 28.2433
Epoch 7/100, Train Loss: 27.8803, Val Loss: 28.2417
Epoch 8/100, Train Loss: 27.9430, Val Loss: 28.2404
Epoch 9/100, Train Loss: 27.9235, Val Loss: 28.2384
Epoch 10/100, Train Loss: 27.8858, Val Loss: 28.2365
Epoch 11/100, Train Loss: 27.8667, Val Loss: 28.2342
Epoch 12/100, Train Loss: 27.8967, Val Loss: 28.2321
Epoch 13/100, Train Loss: 27.9088, Val Loss: 28.2290
Epoch 14/100, Train Loss: 27.9005, Val Loss: 28.2257
Epoch 15/100, Train Loss: 27.8917, Val Loss: 28.2213
Epoch 16/100, Train Loss: 27.8865, Val Loss: 28.2164
Epoch 17/100, Train Loss: 27.8962, Val Loss: 28.2092
Epoch 18/100, Train Loss: 27.8444, Val Loss: 28.1883
Epoch 19/100, Train Loss: 27.7761, Val Loss: 28.1264
Ep

In [120]:
## plot dos pontos de teste e a função f(x) usando o plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_test[:,0], y=x_test[:,1], mode='markers', marker=dict(color=y_test, size=3)))
x = np.linspace(0,5,100)
y = f(x)
fig.add_trace(go.Scatter(x=x, y=y, mode='lines', line=dict(color='orange')))
fig.show()

In [121]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(train_losses)), y=train_losses, mode='lines', name='Train Loss'))
fig.add_trace(go.Scatter(x=np.arange(len(val_losses)), y=val_losses, mode='lines', name='Val Loss'))
fig.update_layout(height=600, title='Train and Val Loss', xaxis_title='Epoch', yaxis_title='Loss')
fig.add_trace(go.Scatter(x=[len(train_losses)], y=[test_loss], mode='markers', marker=dict(color='orange', size=10), name='Test Loss'))
fig.show()